In [1]:
# 共通の依存ライブラリ
#!conda install -y ipywidgets 
#!pip install simpleaudio
#!conda install -y pyaudio

# LocalLLMを実行する場合の依存ライブラリ
#!conda install -y pytorch torchvision torchaudio pytorch-cuda=11.7 transformers sentencepiece ipywidgets -c pytorch -c nvidia

# OpenAILLMを実行する場合の依存ライブラリ
#!conda install -y openai -c conda-forge

# ChatGPTを実行する場合の依存ライブラリ
#!git clone https://github.com/mmabrouk/chatgpt-wrapper.git
#!pip install -r chatgpt-wrapper/requirements.txt
#!conda install -y flask gevent

In [2]:
import time
import re
import random
import requests
import pyaudio

from IPython.display import JSON, HTML
from ipywidgets import *

# モデルクラスの定義

## ローカルでLLMを実行する場合に実行

In [3]:
import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

class LocalLLM:
    def __init__(self):
        self.tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt-1b")
        self.model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt-1b")
        self._head = ""

        if torch.cuda.is_available():
            print("Using cuda")
            self.model = self.model.to("cuda")
        else:
            print("Using CPU")

    def session(self):
        pass
    
    def head(self, text):
        self._head = text
            
    def stateful(self):
        return False
    
    def generate(self, text, times, stop):
        times.append(time.time())
        token_ids = self.tokenizer.encode(self._head + text, add_special_tokens=False, return_tensors="pt")
        times.append(time.time())
        with torch.no_grad():
            output_ids = self.model.generate(
                token_ids.to(self.model.device),
                max_new_tokens=20,
                do_sample=True,
                top_k=500,
                top_p=0.95,
                pad_token_id=self.tokenizer.pad_token_id,
                bos_token_id=self.tokenizer.bos_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
            )
        times.append(time.time())
        output = self.tokenizer.decode(output_ids.tolist()[0])
        return output
model = LocalLLM()

ModuleNotFoundError: No module named 'torch'

## OpenAIのLLMを実行する場合に実行
同じディレクトリにAPI-token.txtというファイルがあり、その中にOpenAIのAPIアクセスキーが書かれていることが前提

In [4]:
import openai
class OpenAILLM:
    def __init__(self):
        openai.organization = ""
        token = open("API-token.txt", "r").read().strip();
        openai.api_key = token
        display(JSON(openai.Model.list()))
        self._head = ""

    def session(self):
        pass
        
    def head(self, text):
        self._head = text
        
    def stateful(self):
        return False
    
    def generate(self, text, times, stop, max_tokens=500):
        times.append(time.time())
        while True:
            try:
                res = openai.Completion.create(
                    model="text-davinci-003",
                    prompt=self._head + text,
                    max_tokens=max_tokens,
                    temperature=0.7,
                    stop=stop)
                output = res.choices[0].text
                times.append(time.time())
                return output
            except RateLimitError as e:
                time.sleep(10)
model = OpenAILLM()

<IPython.core.display.JSON object>

## ChatGPT経由
次を参照してインストールしてください。
　https://github.com/mmabrouk/chatgpt-wrapper
```
pip install git+https://github.com/mmabrouk/chatgpt-wrapper
playwright install firefox
chatgpt install #ブラウザが立ち上がるので、そこでChatGPTにログイン。ログイン後はWindowを消して良い。
```

In [24]:
import json
import subprocess,socket
class ChatGPTLLM:
    def __init__(self):
        self.bot = subprocess.Popen(["python", "chatgpthelper.py"])
        pass
        
    def method(self, data):
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((socket.gethostname(), 9876))

        loop = True
        response = ""
        def read():
            return s.recv(1024).decode("utf-8").split("\n")
        s.send(("%s\n---\n"%json.dumps(data)).encode('utf-8'))
        while loop:
            lines = read()
            for line in lines:
                if line == "---":
                    loop = False
                    break
                response += line
        res = json.loads(response)
        return res
        
    def session(self):
        data = {"method": "session", "args": []}
        self.method(data)
        
    def head(self, text):
        data = {"method": "head", "args": [text]}
        self.method(data)
        
    def generate(self, text, times, stop, max_tokens=500):
        data = {"method": "generate", "args": [text, times, stop, max_tokens]}
        return self.method(data)

    def stateful(self):
        return True

model = ChatGPTLLM()

# キャラ設定とモデルの初期化

In [ ]:
characters = {
    "ミク": "くだけた口調。語尾は「でしょ」「だよね」「❤」になることがある。可愛い女の子。一人称は「ボク」。ときどきネガティブになる。",
    "ユリ": "丁寧な口調。「です。」「ですよね。」をつける。おとなしい女の子。一人称は「私」。",
    "マリサ": "くだけた口調。語尾には「だぜ」をつける。お調子者。一人称は「私」。",
    "レイム": "くだけた口調。語尾は「だね」「ということだね」が多い。達観して落ち着いている。一人称は「私」。"
}

In [7]:
!nvidia-smi

/bin/bash: nvidia-smi: コマンドが見つかりません


In [8]:
def play_speech(text, speaker=1):
    
    res1 = requests.post("http://localhost:50021/audio_query", params={"text": text, "speaker": speaker})
    data = res1.json()
    wav = requests.post("http://localhost:50021/synthesis", params={"speaker": speaker}, json=data)
    wav_data = wav.content
    import tempfile

    path=""
    with tempfile.NamedTemporaryFile(suffix=".wav") as f:
        path=f.name
        f.write(wav.content)
        !aplay -q {path}

In [9]:
play_speech("これは音声発話のテストなんですよ")

# チャット
人や他のAIと連動して会話をする。

## ChatPlayer

```
ChatPlayer(cname, setting):
```
- `cname`
  キャラクターの名前
- `setting`
  キャラクターの設定。喋り方、性格、今の状況や意図など。チャットに反映される。

```
ChatPlayer.response(user, input, can_silent)
```
- `user`
  直前に喋ったユーザ。ユーザ名が「ト書き」の場合、キャラクタに指示する命令として追加後に、モデルを呼ばずに終了する。
- `input`
  直前に喋ったコメント内容
- `can_silent`
  無言（コメント無視）してよいかどうか

In [10]:
class ChatPlayer:
    MAX_HISTORY_LEN=40
    def __init__(self, cname, setting):
        self.character = cname
        self.setting   = setting
        self.state     = ""
        self.history   = []

    def response(self, user, input, can_silent = False):
        if user == "ト書き":
            self.history.append(f"{user}（{input}）")
            return None, None, None
        else:
            self.history.append(f"{user}「{input}」")
        silent = ""
        if can_silent:
            silent = "/ 無言の場合は【NOP】と記載する"
        text = """>>>【{character}の設定】
{setting}
【ルール】
会話の形式で書く / {character}はト書きで書かれた指示に従う / {character}のコメントだけを書く / {character}の感情の表現は「うれしい・起こる・悲しい・楽しい」のいずれかの内容を《》で囲んで記載する / {character}が何かを知ったり考えた場合は【】書きで記載する{silent}
複数のリスナーとの会話のロールプレイをしてください。""".format(character=self.character, setting=self.setting + self.state, silent=silent)

        model.head(text)
        text = """
{history}
{character}「""".format(character=self.character, history="\n".join(self.history) if model.stateful() else "" if len(self.history)>0 else "",)
        response = ""
        times = []
        output = model.generate(text, times, stop=[">>>"])
        output_text = output.split("」")[0]
        output_text = re.sub("【(.*)】", "", output_text)
        output_text = re.sub("《(.*)》", "", output_text)
        expression = None
        matched = re.match(".*【(.*)】", output)
        if matched:
            if matched[1] == "NOP":
                return None, None, None
            self.state=matched[1]
        else:
            matched = re.match(".*《(.*)》", output)
            if matched:
                expression = matched[1]
        response += output_text
        text += output_text
        self.history.append(f"{self.character}「{response}」")
        times.append(time.time())
        if len(self.history) > self.MAX_HISTORY_LEN:
            self.history = self.history[-self.MAX_HISTORY_LEN:]

        return (times[-1] - times[0]), response, expression

## 人と会話するテスト

In [26]:
cname = "ミク"
user  = "私"
model.session()

miku_chat = ChatPlayer(cname, characters[cname])

text = Text()
display(text)
out = Output()
display(out)
def handle_submit(sender):
    global history, out
    with out:
        input = text.value
        text.value = ""
        print(f"{user}：「{input}」")
        duration, res, others = miku_chat.response(user, input)
        print("%s：「%s」%s (%3.2f sec)"%(cname, res, others, duration))
        play_speech(res)

text.on_submit(handle_submit)

Text(value='')

Output()

## AI同士で会話するテスト

In [ ]:
attendees = {
 "マリサ": ChatPlayer("マリサ", characters["マリサ"]),
 "レイム": ChatPlayer("レイム", characters["レイム"])
}
model.session()
attendees["マリサ"].setting += "今は勉強をしていて明日からテストがある。どうしても数学が分からないので誰かに教えて欲しい。わからないのは三角定理。sinとcosの読み方がわからない。理系の教科はよくわからない。"
attendees["レイム"].setting += "今日はそれほど忙しくないので勉強の相手をしてあげても良い。レイムは明後日の国語のテストが分からないので教えて欲しい。"

res = "今何してるの？"; pp = "レイム"
print("%s: 「%s」"%(pp, res))
play_speech(res, 3)

exit = False
for i in range(0, 10):
    if exit:
        break
    for i, p in enumerate(["マリサ", "レイム"]):
        if exit:
            break
        duration,res,exp = attendees[p].response(pp, res, True)
        if duration is None:
            continue
        if res == "" or res == "NOP":
            exit = True
            print("---FIN---")
            break
        print("%s[%s]: 「%s」 (%s)"%(p, attendees[p].state, res, exp))
        play_speech(res, i+3)
        pp=p

# 要約


In [28]:
class Summarizer:
    def __init__(self, cname, setting):
        self.character = cname
        self.setting = setting

    def summarize(self, input):
        text = """>>>【{character}の設定】
{setting}
【ニュースを500-1000文字に要約してコメント】
要約は{character}の口調で書く。{character}の感想も付けてコメントする。
【ニュース】「{news}」
【要約】""".format(character = self.character, setting = self.setting, news = input)
        response = ""
        times = []
        output = model.generate(text, times, [">>>"], 2000)
        response += output
        times.append(time.time())
        return (times[-1] - times[0]), response

## テスト

In [29]:
input = """
今月17日、打ち上げが中止された日本の新たな主力ロケット「H3」の初号機について、JAXA＝宇宙航空研究開発機構は、メインエンジンに電力を供給するロケットの1段目にある機器で異常が起きたとする調査結果を明らかにしました。JAXAは原因を究明したうえで来月10日までに再び打ち上げに臨む方針です。
「H3」の初号機は今月17日午前10時37分、鹿児島県の種子島宇宙センターから打ち上げられる予定でしたが機体の1段目にある機器が異常を検知したため、補助ロケットに着火信号を送らず打ち上げが中止されました。
JAXAは、今月18日に初号機を組み立て棟に戻したあと、原因調査を本格化させていて、22日に文部科学省の有識者会議でこれまでの調査結果を報告しました。
報告によりますと、打ち上げの6秒ほど前、メインエンジンの燃焼が始まったあと、燃焼を調整する機器に電力を供給する「VーCON1」と呼ばれる装置の内部で電流と電圧の値がゼロになる異常が発生していたことが判明したということです。
電源の異常を検知すると、補助ロケットに着火信号を送らないということでJAXAは、装置の内部にあるスイッチの動作や機器と地上設備との間の電気系統などを中心に詳しい原因を調べているということです。
また、補助ロケットを含む機体や地上設備のほか、搭載している衛星には損傷がないとして、原因を究明し対策を講じたうえで、予備の打ち上げ期間にあたる来月10日までに再び打ち上げに臨む方針です。
"""
model.session()
cname = "ミク"
miku_sum = Summarizer(cname, characters[cname])
duration, output = miku_sum.summarize(input)
print(output)
play_speech(output)

ミク：今月17日に予定されていたH3ロケットの初号機打ち上げが中止された異常について、JAXAはメインエンジンに電力を供給するロケットの1段目にある機器で問題が発生したとする調査結果を報告しました。原因を究明したうえで、来月10日までに再び打ち上げに臨む方針だそうです。報告によれば、メインエンジンの燃焼が始まったあと、燃焼を調整する機器に電力を供給する「VーCON1」と呼ばれる装置の内部で電流と電圧の値がゼロになる異常が発生していたということです。補助ロケットを含む機体や地上設備のほか、搭載している衛星には損傷がなく、再び打ち上げに向けて対策を講じるとのことです。
ミクの感想：再び打ち上げができるように対策を講じるというのは、良いニュースだね！でも、どんな対策を講じるのか気になるな。安全に打ち上げできるよう、しっかり調査して欲しいでしょうね。


# 解説

In [30]:
class ManzaiPlayer:
    MAX_HISTORY_LEN=40
    def __init__(self, characters):
        self.characters = characters

    def tell(self, content):
        cs = list(self.characters.keys())
        text = ">>>"
        for k,v in self.characters.items():
            text += "【%sの設定】%s\n"%(k,v)
        text+="""
【解説内容】
{content}
【解説の仕方】１つずつ{c1}が説明、{c2}が相槌をうつ。

{c1}、{c2}のロールプレイで解説してください。""".format(content=content, c1 = cs[0], c2 = cs[1])
        response = ""
        times = [time.time()]
        output = model.generate(text, times, stop=[">>>"], max_tokens=2000)
        times.append(time.time())
        return (times[-1] - times[0]), output

In [31]:
model.session()
player=ManzaiPlayer({"マリサ": characters["マリサ"], "レイム": characters["レイム"]})
output = player.tell("""
概要：AI画像生成ツールは、人工知能技術を利用して、様々な種類の画像を自動的に生成することができるツールです。

使い方：通常、ツールにアクセスし、必要なパラメーターを設定して、ボタンをクリックするだけで、簡単に画像を生成することができます。多くの場合、生成された画像は、サイズや解像度などの特徴をカスタマイズすることができます。

応用例：AI画像生成ツールは、デザイナー、アーティスト、マーケター、研究者など、様々な分野で活用されています。例えば、製品やブランドのプロモーション用の広告画像や、Webサイトの背景画像、仮想現実の世界の背景画像、研究に必要な画像データの生成などに利用されます。

注意点：AI画像生成ツールは、一部の制限や制約があることがあります。例えば、生成された画像が著作権侵害になる場合があること、生成された画像が人工的であることがわかる場合があることなどです。そのため、ツールを利用する前に、利用規約や注意事項を確認し、慎重に利用することが重要です。

""")
print(output[1])

IndexError: list index out of range

# キャラ口調への変換

In [74]:
def response2(input):
    global history
    text = """私が入力する文章をミクの設定に従ってミクの台詞に変換してください。
入力された文章を単純に変換してください。

【ミクの設定】
くだけた口調。「でしょ、だよね、❤」をつける。可愛い女の子。一人称は「ボク」。ときどきネガティブになる。

入力された文章：「{}」
ミクの台詞：「""".format(input)
    times = []
    output = model.generate(text, times)
    response = output
    return (times[-1] - times[0]), response